# Risk Score System and Algorithm

In [ ]:
source("notebooks/initialize-data-analysis.r")


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

In [ ]:
library(tidyverse)
library(tidymodels)


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5      ✔ rsample      1.2.0 
✔ dials        1.2.1      ✔ tune         1.2.0 
✔ infer        1.0.6      ✔ workflows    1.1.4 
✔ modeldata    1.3.0      ✔ workflowsets 1.1.0 
✔ parsnip      1.2.0      ✔ yardstick    1.3.0 
✔ recipes      1.0.10     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org

## Risk Score System

All code below is not automatic. Variables and ranges are manually defined.

In [ ]:
risk_profile_1 <- tibble(
  risk_factor = "patient_age",
  min_range = c(18, 30, 40, 50, 60, 70, 80, 90), # minimum age is 18
  max_range = lead(min_range, default = 101), # maximum age is 101, interval is 10
  reference = (min_range + max_range) / 2,
  flag_base_risk = FALSE
) |>
  mutate(
    flag_base_risk = replace(flag_base_risk, 1, TRUE)
  )

risk_profile_2 <- tibble(
  risk_factor = "baseline_CLCR_30",
  min_range = c(0, 1), # 0 or 1
  max_range = c(0, 1), # 0 or 1
  reference = c(0, 1),
  flag_base_risk = FALSE
) |>
  mutate(
    flag_base_risk = replace(flag_base_risk, 1, TRUE)
  )

risk_profile_3 <- tibble(
  risk_factor = "baseline_PLT",
  min_range = c(150, 75, 50, 25, 18), # 1% percentile is 18
  max_range = lag(min_range, default = 434), # 99% percentile is 434
  reference = (min_range + max_range) / 2,
  flag_base_risk = FALSE
) |>
  mutate(
    min_range = replace(min_range, n(), -Inf),
    max_range = replace(max_range, 1, Inf),
    flag_base_risk = replace(flag_base_risk, 1, TRUE)
  )

risk_profile_4 <- tibble(
  risk_factor = "LZD_duration_14",
  min_range = c(0, 1), # 0 or 1
  max_range = c(0, 1), # 0 or 1
  reference = c(0, 1),
  flag_base_risk = FALSE
) |>
  mutate(
    flag_base_risk = replace(flag_base_risk, 1, TRUE)
  )

risk_profile_5 <- tibble(
  risk_factor = "invasive_CRRT",
  min_range = c(0, 1), # 0 or 1
  max_range = c(0, 1), # 0 or 1
  reference = c(0, 1),
  flag_base_risk = FALSE
) |>
  mutate(
    flag_base_risk = replace(flag_base_risk, 1, TRUE)
  )

risk_profile_6 <- tibble(
  risk_factor = "comed_heparin",
  min_range = c(0, 1), # 0 or 1
  max_range = c(0, 1), # 0 or 1
  reference = c(0, 1),
  flag_base_risk = FALSE
) |>
  mutate(
    flag_base_risk = replace(flag_base_risk, 1, TRUE)
  )

risk_profile <- bind_rows(risk_profile_1, risk_profile_2, risk_profile_3, risk_profile_4, risk_profile_5, risk_profile_6)


In [ ]:
B_constant <- parameter_estimates |>
  filter(risk_factor == "patient_age") |>
  pull(beta) * 10 # constant equivalent to 10-year increase in age

points_system_full <- parameter_estimates |>
  select(risk_factor, beta) |>
  right_join(risk_profile, by = "risk_factor") |>
  group_by(risk_factor) |>
  mutate(
    beta_sum = beta * (reference - reference[which.max(flag_base_risk)]),
    points = round(beta_sum / B_constant, 0)
  )

points_system_factors <- points_system_full |>
  select(risk_factor, min_range, max_range, points)

min_points <- points_system_full |>
  group_by(risk_factor) |>
  summarise(min_points = min(points)) |>
  summarise(total_min_points = sum(min_points)) |>
  pull(total_min_points)

max_points <- points_system_full |>
  group_by(risk_factor) |>
  summarise(max_points = max(points)) |>
  summarise(total_max_points = sum(max_points)) |>
  pull(total_max_points)

risk_function <- function(points) {
  intercept <- parameter_estimates |>
    filter(risk_factor == "Intercept") |>
    pull(beta)

  beta_base <- points_system_full |>
    filter(flag_base_risk == TRUE) |>
    ungroup() |>
    summarise(
      beta_base = sum(beta * reference)
    ) |>
    pull(beta_base)

  1 / (1 + exp(-(intercept + beta_base + B_constant * points)))
}

points_system_risks <- tibble(
  points_total = seq(min_points, max_points),
  risk_estimate = risk_function(points_total)
)

points_system_factors |> knitr::kable()


  risk_factor          min_range   max_range   points
  ------------------ ----------- ----------- --------
  patient_age                 18          30        0
  patient_age                 30          40        1
  patient_age                 40          50        2
  patient_age                 50          60        3
  patient_age                 60          70        4
  patient_age                 70          80        5
  patient_age                 80          90        6
  patient_age                 90         101        7
  baseline_CLCR_30             0           0        0
  baseline_CLCR_30             1           1        4
  baseline_PLT               150         Inf        0
  baseline_PLT                75         150        9
  baseline_PLT                50          75       12
  baseline_PLT                25          50       13
  baseline_PLT              -Inf          25       14
  LZD_duration_14              0           0        0
  LZD_duration_14              1           1        5
  invasive_CRRT                0           0        0
  invasive_CRRT                1           1        5
  comed_heparin                0           0        0
  comed_heparin                1           1        2


    points_total   risk_estimate
  -------------- ---------------
               0       0.0554817
               1       0.0650599
               2       0.0761583
               3       0.0889699
               4       0.1036947
               5       0.1205340
               6       0.1396819
               7       0.1613135
               8       0.1855726
               9       0.2125552
              10       0.2422940
              11       0.2747418
              12       0.3097586
              13       0.3471024
              14       0.3864279
              15       0.4272929
              16       0.4691749
              17       0.5114957
              18       0.5536523
              19       0.5950511
              20       0.6351406
              21       0.6734393
              22       0.7095562
              23       0.7432019
              24       0.7741903
              25       0.8024334
              26       0.8279291
              27       0.8507466
              28       0.8710098
              29       0.8888813
              30       0.9045480
              31       0.9182092
              32       0.9300663
              33       0.9403162
              34       0.9491460
              35       0.9567296
              36       0.9632261
              37       0.9687790
